# <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:120%;text-align: left'>Régression linéaire</div></b>

In [50]:
import pandas as pd, numpy as np, seaborn as sns, warnings, os, sys, time, copy as cp
from datetime import datetime as dt
from matplotlib import pyplot as plt

import matplotlib.font_manager as fm
import plotly.express as px
import plotly.graph_objs as go
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

font1 = fm.FontProperties(size=20)
font2 = fm.FontProperties(size=24)

warnings.filterwarnings(action="ignore")

if int(str(sns.__version__).split('.')[1]) > 8 : 
    plt.style.use('seaborn-v0_8-darkgrid')
else:
    plt.style.use('seaborn-darkgrid')
    # plt.style.use('seaborn-poster')
sns.set(font_scale=3)
np.random.seed(123456789)

## Découpage du jeu de données 

### Importation des données

In [51]:
donnees = pd.read_csv('50_Startups.csv')

### Nettoyage des données

In [52]:
# Vérifier les valeurs manquantes
missing_values = donnees.isnull().sum()

# Vérifier les types de données
data_types = donnees.dtypes

missing_values, data_types

(R&D Spend          0
 Administration     0
 Marketing Spend    0
 State              0
 Profit             0
 dtype: int64,
 R&D Spend          float64
 Administration     float64
 Marketing Spend    float64
 State               object
 Profit             float64
 dtype: object)

### Conversion en valeur numérique

In [53]:
df_encoded = pd.get_dummies(donnees, columns=["State"], drop_first=True)

X = df_encoded.drop(columns=["Profit"])
y = df_encoded["Profit"]

X.head()

,R&D Spend,Administration,Marketing Spend,State_Florida,State_New York
0,165349.20,136897.80,471784.10,False,True
1,162597.70,151377.59,443898.53,False,False
2,153441.51,101145.55,407934.54,True,False
3,144372.41,118671.85,383199.62,False,True
4,142107.34,91391.77,366168.42,True,False


### Standardisation des données

In [54]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

X_scaled_df.head()

,R&D Spend,Administration,Marketing Spend,State_Florida,State_New York
0,2.016411,0.560753,2.153943,-0.685994,1.393261
1,1.955860,1.082807,1.923600,-0.685994,-0.717741
2,1.754364,-0.728257,1.626528,1.457738,-0.717741
3,1.554784,-0.096365,1.422210,-0.685994,1.393261
4,1.504937,-1.079919,1.281528,1.457738,-0.717741


## Division en train & test

In [65]:
# Diviser les données en ensembles d'entraînement (80%) et de test (20%)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Vérifier la taille des ensembles
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((40, 5), (10, 5), (40,), (10,))

### Analyse en composante principale

In [64]:
# Initialiser l'ACP pour conserver suffisamment de variance (ex: 95%)
pca = PCA(n_components=0.95)

# Appliquer l'ACP sur les données d'entraînement
X_train_pca = pca.fit_transform(X_train)

# Appliquer l'ACP sur les données de test en utilisant le même modèle
X_test_pca = pca.transform(X_test)

# Vérifier le nombre de composantes principales retenues
X_train_pca.shape, X_test_pca.shape


((40, 4), (10, 4))

### Mettre la colone profils sur une échelle logarithmique pour diminuer l'écart avec les colonnes centré réduite

# <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:120%;text-align: left'>Régression Linéaire</div></b>

Voici le code équivalent lorsqu’on utilise **Scikit-Learn**

L’équation normale calcule l’inverse de $X^{T} ⋅ X$, qui est une matrice $(n + 1) × (n + 1)$ (où n est le nombre de variables). La complexité algorithmique d’une inversion de matrice de taille n × n se situe entre $O(n^{2,4})$ et $O(n^{3})$, selon l’algorithme d’inversion utilisé. Autrement dit, si vous doublez le nombre de variables, le temps de calcul est grosso modo multiplié par un facteur compris entre $2^{2,4} = 5,3$ et $2^{3} = 8$.

In [57]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X, y)
lin_reg.intercept_, lin_reg.coef_

(50125.34383160422,
 array([ 8.06023114e-01, -2.70043196e-02,  2.69798610e-02,  1.98788793e+02,
        -4.18870191e+01]))

## <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:120%;text-align: left'>Choix de la métrique de performance</div></b>

### <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:120%;text-align: left'>Erreur moyenne absolute</div></b>

$MAE = \frac{1}{n} \sum_{i=1}^n \left| y_i - f(x_i)\right|$

In [58]:
from sklearn.metrics import median_absolute_error
median_absolute_error(y, lin_reg.predict(X))

5050.731847936986

### <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:120%;text-align: left'>Erreur quadratique moyenne(mean absolute error)</div></b>

$MSE = \frac{1}{n} \sum_{i=1}^n (y_i-f(x_i))^2$

In [59]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y, lin_reg.predict(X))

6475.500708609342

### <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:120%;text-align: left'>Racine quarrée de l'erreur quadratique moyenne(mean squared error)</div></b>

$RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^n (y_i-f(x_i))^2}$

In [60]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y, lin_reg.predict(X))

78406792.88803767


### <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:120%;text-align: left'>Erreur logarithmique quadratique moyenne(mean squared logarithmic error)</div></b>


$RMSLE = \sqrt{\frac{1}{n} \sum_{i=1}^n (\log(y_i+1) - \log(f(x_i)+1))^2}$

In [61]:
from sklearn.metrics import mean_squared_log_error
mean_squared_log_error(y, lin_reg.predict(X))

0.036459236616758074

### <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:120%;text-align: left'>$R^{2}$ coefficient de détermination linéaire de Pearson</div></b>

$RSE = \frac{\sum_{i=1}^n (y_i - f(x_i))^2}{\sum_{i=1}^n (y_i - \bar{y})^2}$ avec $\bar{y} = \frac{1}{n} \sum_{i=1}^n y_i$  

$R = \frac{\sum_{i=1}^n (y_i - \bar y) (f(x_i) - \overline{f(x)})}{\sqrt{\sum_{i=1}^n (y_i - \bar y)^2} \sqrt{\sum_{i=1}^n (f(x_i) - \overline{f(x)})^2 }}$

In [62]:
from sklearn.metrics import r2_score
print(r2_score(y, lin_reg.predict(X)))
print(r2_score(y, lin_reg.predict(X), multioutput='variance_weighted'))
print(r2_score(y, lin_reg.predict(X), multioutput='uniform_average'))
print(r2_score(y, lin_reg.predict(X), multioutput='raw_values'))
print(np.sqrt(r2_score(y, lin_reg.predict(X))))

0.9507524843355148
0.9507524843355148
0.9507524843355148
[0.95075248]
0.9750653743906174
